### Introduction

In this notebook we investigate accuracy and performance of the decomposition
method, using a benchmark which appeared in [1].

This is an 8-dimensional helicopter model with a 20-dimensional controller,
taken from [2].

The configuration files for this model can be obtained from the HSCC2017 repeatibility
evaluation. The source code of HyLAA is available on Github.

---

*References*:

[1] [HyLAA: A Tool for Computing Simulation-Equivalent Reachability for Linear
Systems. S. Bak, P. S. Duggirala. HSCC 2017](http://stanleybak.com/papers/bak2017hscc.pdf)

[2] Multivariable Feedback Control: Analysis and Design. John Wiley & Sons, 2005.
S. Skogestad and I. Postlethwaite. 

### Accuracy evaluation

The model coeffiecients and the set of initial conditions have been defined in the file `helicopter.jl`. In particular:

- it has no inputs,
- it is 28-dimensional,
- the initial state for this system is the product of intervals; thus, we can define it as `Hyperrectangle` type provided by the `LazySets` module.

Now, let's load the model file and our package:

In [ ]:
include("helicopter.jl")

First, we run the experiment from Fig. 3 of [1]. This is a plot of the variable $x_8$ over time. Since we use blocks of size 2, the corresponding block number is 4 (use the `@block_id` macro as a shortcut).

The step size is $\delta = 0.1$, and the time horizon is 30sec. Hence, the number of discrete steps that we need is $N=300$. This benchmark problem can be run with the `compute` function, where we pass the plot variables ($0$ being the time variable).

By default, we compute in dense time.

In [ ]:
compute(300, 0.1, :block=>4, :plot_vars=>[0, 8], :plot_backend=>"pyplot_savefig")

![helicopter1](helicopter1.png)

The reachability computation of these 300 time steps takes around 0.08 seconds (there is ~2% variation between runs). The speed of the other tools is not reported in [1], however, to judge for Fig. 2(a), is also around 1 second. Later we see how it behaves with an extended model.

The shape of the plot conforms to those in [1], Fig. 3, and in terms of accuracy, it is very similar to that of HyLAA, hence better than both SpaceEx results reported there.

![heli_bench_x8](heli_bench_x8.png)

Above we have used a "continous" reachability, which is the same as in SpaceEx, and that guarantees that all trajectories of the continuous system belong to the union of overapproximating sets. 

However, HyLAA computes the covering of the discretized system, i.e. no bloating at all of the initial states is used.

In our tool, the flag `ct_bloating_factor`, set to `true` by default (continuous system), can be used to define either one or the other case. 

First, let's consider the effect of the discretization for the first 10 reachable sets:

In [ ]:
compute(10, 0.1, :block=>4, :plot_vars => [0, 8], :ct_bloating_factor=>true, :plot_backend=>"pyplot_savefig",
        :plot_name=>"heli2.png")

![heli2](heli2.png)

Now let's do the "discrete" case (corresponding to the plot in HyLAA):

In [ ]:
compute(10, 0.1, :block=>4, :plot_vars=>[0, 8], :ct_bloating_factor=>false, :plot_backend=>"pyplot_savefig",
        :plot_name=>"heli3.png")

![heli3](heli3.png)

The initial set is of course the same in both cases, by definition. But the second set is larger with bloating, as expected. It is however a small amount (with bloating is 0.22, and without bloating it is 0.15, approximatively). Observe also that the width of the last computed set is printed in the results: 

In [ ]:
width_bloating = 4.490740e-01 - (-4.490740e-01)
width_no_bloating = 3.683577e-01 - (-3.683577e-01)
width_bloating, width_no_bloating

## Safety property

In [ ]:
safety_property = Property([1., 0.], 0.12) # x1 < 0.12

In [ ]:
600 * 0.05

In [ ]:
# in dense time
compute(400, 0.1, :block=>1, :plot_vars=>[0, 1], 
        :plot_backend=>"pyplot_savefig", :plot_name=>"helicheck.png")

![helicheck](helicheck.png)

In [ ]:
compute(300, 0.1, :mode=>"check", :property => safety_property)
#:property => Property([1., 0.], 0.12), # x1 < 0.12
#:blocks => 1:14,
#:mode => "reach",
#:algorithm => "explicit_block",

## Performance 

In order to analyze scalability, in [1] the helicopter coefficients matrix is
replicated `ncopies` times, withing the same model. Inspecting the SpaceEx configuration
files, one sees that the enlarged model consists in non-coupled copies of the helicopter,
and the initial states of each copy are all of them equal as the one used above.

It is easy to create copies of the Helicopter model in Julia. See the Appendix to this notebook for a step-by-step explanation. 

Since the original model is 28-dimensional, the number of variables of a
q-th order model is a multiple of 28. To obtain some scalability plots,
let's choose powers of two.

xyzxyz

In [ ]:
compute(300, 0.1, :case => "spaceexIC_no_input_ncopies", :case_ncopies => 20)

The performance results are summarized in the table below:

|order|vars|
|----|----|
|1|28|
|2|56|
|4|112|
|8|224|
|16|448|
|32|896|
|64|1792|

---

*This is the end of this notebook investigating the reachability problem of the helicopter model.
This notebook was prepared by [Marcelo Forets](http://marcelo-forets.fr).

*For corrections, remarks, etc., please send them to `mforets@gmail.com`. Enjoy! *

### Appendix: making the replicated model

The function `blkdiag` produces a block-diagonal matrix built from the matrices that receives
as argument. It only works with sparse matrices, and the arguments are one or more
matrices, in comma-separated fashion. Finally, recall that the splat
operator, `...`, is handy to transform a list into a list of arguments.

Let's see this in an example. Suppose that we want to make `ncopies = 3` copies
of a matrix A. We can do itas follows: 

In [ ]:
A = randn(2, 2)

In [ ]:
A = blkdiag([sparse(A) for i in 1:3]...); full(A)

The initial states were given as a `Hyperrectangle`. A simple way of defining
the enlarged initial set, `X0`, is to use a cartesian product. We can do it
using list comprehension as follows:

In [ ]:
n = 28
X0_single = Hyperrectangle(zeros(n), [fill(0.1, 8); zeros(20)])

In [ ]:
ncopies = 2
X0_replicated = CartesianProductArray([X0_single for i in 1:ncopies])